In [1]:
import sqlite3
import json

In [ ]:
all = set()
with open('../data/test_entity_data.json', 'r') as f:
    t = json.loads(f.read())
    print(len(t))
    for item in t:
        id = item[0]
        name = item[1][0]
        instance = item[1][1]
        ar = item[2].get('ar', '')
        de = item[2].get('de', '')
        es = item[2].get('es', '')
        fr = item[2].get('fr', '')
        it = item[2].get('it', '')
        ja = item[2].get('ja', '')

        tup = (id,name,instance, ar, de, es, fr, it, ja)
        all.add(tup)


6641


{('Q369547',
  'Sázava Monastery',
  'monastery',
  '',
  'Kloster Sázava',
  'Monasterio de Sázava',
  'monastère de Sázava',
  'Monastero di Sázava',
  ''),
 ('Q779815',
  'animation director',
  'filmmaking occupation',
  'مخرج رسوم متحركة',
  'Regisseur für Animation',
  'director de animación',
  'chef-animateur',
  "direttore dell'animazione",
  '作画監督'),
 ('Q863438',
  'The Valley of Fear',
  'literary work',
  'وادي الخوف',
  'Das Tal der Angst',
  'El valle del terror',
  'La Vallée de la peur',
  'La valle della paura',
  '恐怖の谷'),
 ('Q853935',
  'Kocatepe Mosque',
  'mosque',
  'جامع خوجه تبه',
  'Kocatepe-Moschee',
  'Mezquita Kocatepe',
  'mosquée Kocatepe',
  '',
  'コジャテペ・モスク'),
 ('Q506505',
  'The Comedy of Errors',
  'dramatic work',
  'كوميديا الأخطاء',
  'Die Komödie der Irrungen',
  'La comedia de las equivocaciones',
  'La Comédie des erreurs',
  'La commedia degli errori',
  '間違いの喜劇'),
 ('Q13923',
  'The Grim Adventures of Billy & Mandy',
  'animated television serie

In [31]:
conn = sqlite3.connect('../database/entities.db')
cursor = conn.cursor()

cursor.execute('PRAGMA table_info(entity_translation);')
cursor.fetchall()
conn.close()

In [32]:
conn = sqlite3.connect('../database/ent.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE entity_translation(
                    wiki_id PRIMARY KEY,
                    name NOT NULL,
                    instance_of,
                    ar,
                    de,
                    es,
                    fr, 
                    it,
                    ja 
               )''')


In [33]:
for item in all:
    try:
        cursor.execute("""INSERT INTO entity_translation (wiki_id, name, instance_of, ar, de, es, fr, it, ja) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, item)
    except sqlite3.IntegrityError:
        print(item, "In db already")

conn.commit()

In [28]:
cursor.execute('SELECT * FROM entity_translation')
cursor.fetchall()

[('Q369547',
  'Sázava Monastery',
  'monastery',
  '',
  'Kloster Sázava',
  'Monasterio de Sázava',
  'monastère de Sázava',
  'Monastero di Sázava',
  ''),
 ('Q779815',
  'animation director',
  'filmmaking occupation',
  'مخرج رسوم متحركة',
  'Regisseur für Animation',
  'director de animación',
  'chef-animateur',
  "direttore dell'animazione",
  '作画監督'),
 ('Q863438',
  'The Valley of Fear',
  'literary work',
  'وادي الخوف',
  'Das Tal der Angst',
  'El valle del terror',
  'La Vallée de la peur',
  'La valle della paura',
  '恐怖の谷'),
 ('Q853935',
  'Kocatepe Mosque',
  'mosque',
  'جامع خوجه تبه',
  'Kocatepe-Moschee',
  'Mezquita Kocatepe',
  'mosquée Kocatepe',
  '',
  'コジャテペ・モスク'),
 ('Q190135',
  'The Little Mermaid',
  'animated film',
  'حورية البحر',
  'Arielle, die Meerjungfrau',
  'La sirenita',
  'La Petite Sirène',
  'La sirenetta',
  'リトル・マーメイド'),
 ('Q183259',
  'Super Nintendo Entertainment System',
  'electronic device model',
  'سوبر نينتندو إنترتينمنت سيستم',
  'Su

In [34]:
def command(query_string:str)->list[tuple]:
    """Raw SQL Executor
    
    ARGS
        **query_string** -- A string of raw sql to execute on the database 

    RETURNS
        A list of results
    NOTES
        Will literally let you do anything be careful
        It opens and closes the SQL connection so you do not have to 
    """
    con = sqlite3.connect('../database/ent.db')
    cur = con.cursor()

    res = cur.execute(query_string)
    res = res.fetchall()

    con.close()

    return res


In [35]:
def query_by_name(name:str, projections='*', additional='')->list[tuple]:
    """Query over the table specifically by name
    
    ARGS
        **name**        -- Literal name or pattern to match against for names 
        **projections** -- Columns you would like projected (default all columns are returned)
        **additional**  -- Any additional SQL you want to add on to the end of the function call (default none)

    RETURNS
        A list of query results
    EXAMPLES
        1. Query a particular name: `query_by_name("John Smith")` -> [(wiki_id, name, instance_of, ... translations)]
        2. Query over a pattern: `query_by_name("Jo%")` -> [(names that start with 'Jo')]
        3. Query over a pattern projecting only the name and arabic translation: `query_by_name`("Jo%", "name, ar")
    """
    res = command(f'''
        SELECT {projections} FROM entity_translation WHERE name like "{name}" {additional}
    ''')

    return res

def query_by_id(ids:list, projections='*', additional='')->list[tuple]:
    """Perform a query by passing in a list of wiki_ids
    
    ARGS
        **ids** -- A list of wiki_ids to query for i.e ['Q49', 'Q1234']
        **projections** -- A string of columns to project (default all)
        **additional** -- Additional sql to pass into the request (default none)

    RETURNS
        A list of tuples representing the query results 
    """
    ids = [f'\"{x}\"' for x in ids] #annoying requirement to wrap the ids in quotes
    q_string = f'''SELECT {projections} FROM entity_translation WHERE wiki_id IN ({", ".join(ids)}) {additional}'''
    res = command(q_string)

    return res

In [45]:
query_by_id(["Q62029"])
query_by_name('Friedrich Wilhelm, Duke of Brunswick-Wolfenb\u00fcttel')

[('Q62029',
  'Friedrich Wilhelm, Duke of Brunswick-Wolfenbüttel',
  'human',
  'فريدرخ فيلهلم دوق براونشفايغ-فولفنبوتل',
  'Friedrich Wilhelm I, Herzog von Braunschweig',
  'Federico Guillermo de Brunswick-Wolfenbüttel',
  'Frédéric-Guillaume de Brunswick-Wolfenbüttel',
  'Federico Guglielmo di Brunswick',
  'フリードリヒ・ヴィルヘルム')]